# Pinecone Vector Database Integration

**References:**

    - [LanChain Pinecone](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)
    - [PineconeVectorStore API](https://api.python.langchain.com/en/latest/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html)

In [6]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from loaders.text_loaders import TextLoaderWithMetadata
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore



In [2]:
load_dotenv("../.env")

True

In [13]:
# prepare modules for document extraction
loader = TextLoaderWithMetadata(
    "/Users/tjosh/codes/axum/druginsights/data/BNFPaeds_prettified.txt"
)
documents = loader.load(
    metadata={
        "title": "British National Formulary for Children",
        "link": "https://www.nice.org.uk/bnfc-uk-only",
        "references": [
            "NICE. (n.d.). BNFc is only available in the UK. https://www.nice.org.uk/bnfc-uk-only"
        ],
    },
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, chunk_overlap=0
)
docs = text_splitter.split_documents(documents)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

len(docs)

3693

In [14]:
docs[0]

Document(page_content="name='Amoxicillin' diseases_and_usecases='Bacterial infections such as pneumonia, bronchitis, and infections of the ear, nose, throat, skin, and urinary tract.' warnings='Do not use if allergic to penicillin. Use with caution in patients with a history of liver or kidney disease. Long-term use may result in a secondary infection.' indications='Treatment of bacterial infections. Effective against susceptible strains of Streptococcus, Staphylococcus, and Haemophilus species.' contraindications='Hypersensitivity to amoxicillin, penicillin, or any component of the formulation. History of a severe allergic reaction (e.g., anaphylaxis) to another beta-lactam antibiotic (e.g., cephalosporin, carbapenem).' dosages_and_how_it_should_be_taken='Adults: 250-500 mg every 8 hours or 500-875 mg every 12 hours. Children: 20-40 mg/kg/day in divided doses every 8 hours or 25-45 mg/kg/day in divided doses every 12 hours. The exact dosage depends on the type and severity of the infe

In [3]:
# Prepare pinecone

PC = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
PC_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]
PC.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'axum-druginsights-80ua1d8.svc.eastus2-5e25.prod-azure.pinecone.io',
              'metric': 'cosine',
              'name': 'axum-druginsights',
              'spec': {'serverless': {'cloud': 'azure', 'region': 'eastus2'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [8]:
# run this only once, if the index is not created

# PC.create_index(
#             name=PC_INDEX_NAME,
#             dimension=1536,
#             metric="cosine",
#             spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#         )

In [15]:
# Adding documents to the vector database. Be careful, this will overwrite the previous documents.
# If you want to add more documents, you can use the .add_documents method instead.
# run only once for the same documents
vectorstore = PineconeVectorStore.from_documents(docs, embeddings, index_name=PC_INDEX_NAME)

# Or, load an existing vector store
# vectorstore = PineconeVectorStore(embedding=embeddings, index_name=PC_INDEX_NAME)


In [11]:
# let us test out the document search

query = "ACE inhibitors"
docs = vectorstore.similarity_search(query, k=3)
print(len(docs))
print(docs[0].page_content)

3
name='ACE inhibitors' diseases_and_usecases='Used in the management of diabetic nephropathy, hypertension, and proteinuria in children with nephritis.' warnings='ACE inhibitors can cause a very rapid fall in blood pressure in volume-depleted children; treatment should therefore be initiated with very low doses. Adverse effects such as apnoea, seizures, renal failure, and severe unpredictable hypotension are very common in the first month of life and it is therefore recommended that ACE inhibitors are avoided whenever possible, particularly in preterm neonates.' indications='Indicated for the management of hypertension and proteinuria in children with nephritis and for the treatment of diabetic nephropathy.' contraindications='Contraindicated in children known to have severe bilateral renal artery stenosis or severe stenosis of the artery supplying a single functioning kidney. Also contraindicated in children with known or suspected renovascular disease, unless the blood pressure cann

In [1]:
import json
import os

os.chdir("../")


In [3]:
from src.agents.prompts.qa_prompts_2 import CONTEXTUALIZE_Q_SYSTEM_PROMPT, QA_SYSTEM_PROMPT

In [4]:
with open("config.json", "r") as f:
    config = json.load(f)

config["contextualize_q_system_prompt"] = CONTEXTUALIZE_Q_SYSTEM_PROMPT
config["qa_system_prompt"] = QA_SYSTEM_PROMPT

with open("config.json", "w") as f:
    json.dump(config, f)
